In [1]:
#import some packages
import cv2, io, json, logging, os, sys, tempfile
import numpy as np
from PIL import Image

In [ ]:
%run Common.ipynb
%run CustomVision.ipynb

In [4]:
#Specify the time (in seconds) you would like frames to be extract from the video below. Useful if the fish videos start on the boat and that footage is unwanted for labelling. 
def extract_frames_from_video(file_path, custom_vision_training_key, custom_vision_endpoint):
    print('Processing {0}...'.format(file_path))
    
    parts = file_path.split(os.sep)

    video_name = parts[-1]
    print(video_name)

    location_name = parts[-2]
    print(location_name)

    transect_name = parts[-3]
    print(transect_name)

    channel_name = parts[-4]
    print(channel_name)

    year = parts[-5]
    print(year)

    video_name = os.path.join(year, channel_name, transect_name, location_name).replace(os.sep, '-').replace(' ', '-')

    starting_time = 0 # Seconds...
    print(starting_time)

    video_capture = cv2.VideoCapture(file_path)
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    print(fps)

    frame_count = int(starting_time * fps)
    print(frame_count)

    video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_count)

    success, np_image = video_capture.read()

    images = []

    while success:
        if (frame_count % fps) == 0:
            frame_name = '{0}_{1}.jpg'.format(video_name, frame_count)
            print('Adding {0}...'.format(frame_name))

            np_image = cv2.cvtColor(np_image, cv2.COLOR_BGR2RGB)

            buffer = io.BytesIO()

            Image.fromarray(np_image).save(buffer, format='JPEG')

            images.append(ImageFileCreateEntry(name=frame_name, contents=buffer.getvalue()))
            
            if len(images) == 64:
                upload(custom_vision_project_id, images, custom_vision_training_key, custom_vision_endpoint)
                images = []
                
        frame_count += 1

        success, np_image = video_capture.read()

    # Upload the remainder...
    upload(custom_vision_project_id, images, custom_vision_training_key, custom_vision_endpoint)
    images = []
                
    print('Done.')

In [5]:
def upload(custom_vision_project_id, images, custom_vision_training_key, custom_vision_endpoint):
    print('Uploading {0} frames...'.format(len(images)))
    result = create_images_from_files(custom_vision_project_id, images, custom_vision_training_key, custom_vision_endpoint)

In [6]:
#Specify where the videos are stored on your local desktop
video_path = '<insert path here e.g. C:\fishvideos\transect 1\>'

In [7]:
#check the file type and change here e.g. .MP4 (file type is case sensitive)
video_names = [video_name for video_name in os.listdir(video_path) if '.MP4' in video_name]

In [ ]:
for video_name in video_names:
    extract_frames_from_video(os.path.join(video_path, video_name), custom_vision_training_key, custom_vision_endpoint)